In [1]:
import tensorflow as tf
import datetime


if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [3]:
df = pd.read_csv('FinalList.csv')
df

,ID,Fuel,Class,Sub-Class,Group,BP2,Uncertainty,Source,SMILES,BP,...,HeteroA,AromaticR,CPS,GATS2m,FPSA3,WNSA3,PNSA3,ATS3p,RPCG,RNCG
0,1,"1,3,5-trimethylbenzene",Hydrocarbons,Aromatic hydrocarbons,Monocyclic aromatic hydrocarbons,437.908,0.182452,NIST,Cc1cc(C)cc(C)c1,437.872222,...,0,6,119.007881,0.810185,0.023771,-1.532920,-4.779854,58.506429,0.142741,0.128051
1,2,1-Butene,Hydrocarbons,Aliphatic hydrocarbons,Alkenes,266.844,0.244792,NIST,CCC=C,267.038889,...,0,0,97.357465,1.260417,0.026443,-1.232569,-5.459298,16.143384,0.192906,0.349211
2,3,1-Pentene,Hydrocarbons,Aliphatic hydrocarbons,Alkenes,303.102,0.065615,NIST,CCCC=C,303.038889,...,0,0,128.152792,1.312500,0.026643,-1.308787,-5.161937,25.164912,0.163565,0.296013
3,4,"2,3-Dimethylpentane",Hydrocarbons,Aliphatic hydrocarbons,Alkanes,362.941,0.395675,NIST,CCC(C)C(C)C,363.150000,...,0,0,185.939758,1.183248,0.019681,-1.059327,-3.541765,58.142762,0.077032,0.165887
4,5,"2,5-dimethylhexane",Hydrocarbons,Aliphatic hydrocarbons,Alkanes,382.231,0.067620,NIST,CC(C)CCC(C)C,382.261111,...,0,0,207.363945,1.222512,0.019948,-1.227298,-3.733328,65.151442,0.066477,0.140495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,321,trans-2-hexene,Hydrocarbons,Aliphatic hydrocarbons,Alkenes,341.005,0.014359,NIST,CCC/C=C/C,341.020000,...,0,0,157.041260,1.275000,0.023728,-1.179628,-4.126941,32.183671,0.150380,0.240837
321,322,trans-crotonic acid,Carboxylic acids and salts,Aliphatic carboxylic acids,Alkenylcarboxylic acids,455.644,1.544870,NIST,C/C=C/C(O)=O,458.150000,...,2,0,14.829893,0.720190,0.045617,-7.854970,-31.651760,15.531601,0.384312,0.561076
322,323,trimethylamine,Amines,Aliphatic amines,Tertiary aliphatic amines,276.068,0.240339,NIST,CN(C)C,276.020000,...,1,0,109.058920,1.176416,0.029365,-0.763550,-3.439881,20.043798,0.111111,0.881493
323,324,valeraldehyde,Aldehydes,Aliphatic aldehydes,Alkyl aldehydes,375.811,0.150969,NIST,CCCCC=O,376.150000,...,1,0,121.591801,1.154950,0.029459,-4.750226,-17.386313,28.463014,0.253451,0.643550


In [4]:
keep = ['BP2','Wiener','WNSA3','GATS1v','DM','BalabanJ','HBondD']
df2 = df[keep]
df2

,BP2,Wiener,WNSA3,GATS1v,DM,BalabanJ,HBondD
0,437.908,84,-1.532920,1.111111,0.037601,2.340852,0
1,266.844,10,-1.232569,1.500000,0.336790,1.974745,0
2,303.102,20,-1.308787,1.500000,0.328765,2.190610,0
3,362.941,46,-1.059327,1.642857,0.056010,3.144208,0
4,382.231,74,-1.227298,1.625000,0.078138,2.927819,0
...,...,...,...,...,...,...,...
320,341.005,35,-1.179628,1.500000,0.075640,2.339092,0
321,455.644,32,-7.854970,1.147885,1.794456,2.627215,1
322,276.068,9,-0.763550,1.934765,0.575520,2.323790,0
323,375.811,35,-4.750226,1.491124,2.883166,2.339092,0


In [5]:
# features = df2[df2.columns[1:]]
# input_dim = features.shape[1]
# input_dim
#The size of a batch must be more than or equal to one and less than or 
#equal to the number of samples in the training dataset
# len(x_train)

In [6]:
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV,KFold

x = df2.drop(['BP2'],axis = 1).to_numpy()
y = df2['BP2'].values
x_train, x_test, y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 0)

In [7]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [8]:
# input_dim = x_train.shape[1]

In [9]:
from keras.wrappers.scikit_learn import KerasRegressor


def create_model(layers,loss,drop,kernel_init):
    model= Sequential()
    for i, nodes in enumerate(layers):
        if i ==0:
            model.add(Dense(nodes,input_dim=x_train.shape[1],
                            kernel_initializer=kernel_init, activation='relu'))
            model.add(Dropout(drop))
        else:
            model.add(Dense(nodes,kernel_initializer=kernel_init, activation='relu'))
            model.add(Dropout(drop))
    model.add(Dense(1,kernel_initializer=kernel_init)) #No activation beyond this point
    model.compile(loss = loss, optimizer='adam')
    return model

# def model2(input_dim, loss, r1, l1, r2, l2):
#     """
#     Returns a two layer artificial neural network model with the number of units
#     defined by l1 and l2, and regularization coefficients by r1 and r2.
#     """
#     model = Sequential()
#     model.add(Dense(l1, input_dim=input_dim, kernel_initializer='he_uniform', activation='relu'))
#     model.add(Dropout(r1))
#     model.add(Dense(l2, kernel_initializer='he_uniform', activation='relu'))
#     model.add(Dropout(r2))
#     model.add(Dense(1, kernel_initializer='he_uniform'))
#     model.compile(loss=loss, optimizer='adam')
#     return model

model = KerasRegressor(build_fn=create_model,verbose=0)





In [10]:
# Define search space
layers = [[220,220],[200,180],[300,250,200]]
kernels = ['he_uniform','normal','he_normal']
epochs = [5000,7000,10000,15000]
batch_size = [4, 8, 16, 32, 64, 128]
loss = ['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'mean_squared_logarithmic_error']
drop = [0.4,.45,.50]
#Define parameters
param_grid = dict(layers=layers,kernel_init=kernels,batch_size=batch_size, epochs=epochs, loss=loss,
                  drop=drop)



In [11]:

grid = RandomizedSearchCV(model,param_grid,
                    cv=KFold(n_splits=10, shuffle=True, random_state=9),
                    verbose=2, scoring='neg_mean_squared_error',random_state=43)
grid.fit(x_train,y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] END batch_size=64, drop=0.4, epochs=15000, kernel_init=he_normal, layers=[220, 220], loss=mean_squared_error; total time= 2.6min
[CV] END batch_size=64, drop=0.4, epochs=15000, kernel_init=he_normal, layers=[220, 220], loss=mean_squared_error; total time= 2.3min
[CV] END batch_size=64, drop=0.4, epochs=15000, kernel_init=he_normal, layers=[220, 220], loss=mean_squared_error; total time= 2.4min
[CV] END batch_size=64, drop=0.4, epochs=15000, kernel_init=he_normal, layers=[220, 220], loss=mean_squared_error; total time= 2.5min
[CV] END batch_size=64, drop=0.4, epochs=15000, kernel_init=he_normal, layers=[220, 220], loss=mean_squared_error; total time= 2.4min
[CV] END batch_size=64, drop=0.4, epochs=15000, kernel_init=he_normal, layers=[220, 220], loss=mean_squared_error; total time= 2.5min
[CV] END batch_size=64, drop=0.4, epochs=15000, kernel_init=he_normal, layers=[220, 220], loss=mean_squared_error; total time= 2.5min

RandomizedSearchCV(cv=KFold(n_splits=10, random_state=9, shuffle=True),
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x00000220AF257588>,
                   param_distributions={'batch_size': [4, 8, 16, 32, 64, 128],
                                        'drop': [0.4, 0.45, 0.5],
                                        'epochs': [5000, 7000, 10000, 15000],
                                        'kernel_init': ['he_uniform', 'normal',
                                                        'he_normal'],
                                        'layers': [[220, 220], [200, 180],
                                                   [300, 250, 200]],
                                        'loss': ['mean_squared_error',
                                                 'mean_absolute_error',
                                                 'mean_absolute_percentage_error',
                                                 'mean_squared_logarithmic_error'

In [12]:
print(grid.best_params_)
print(grid.best_score_)

{'loss': 'mean_squared_error', 'layers': [220, 220], 'kernel_init': 'he_normal', 'epochs': 15000, 'drop': 0.4, 'batch_size': 64}
-263.047916920613


In [13]:
from sklearn.metrics import r2_score
y_train_pred = grid.predict(x_train)
y_test_pred = grid.predict(x_test)
print('R^2 train: %.3f, test: %.3f' %
       (r2_score(y_train,y_train_pred),
        r2_score(y_test, y_test_pred)))

R^2 train: 0.982, test: 0.928


In [14]:
from sklearn.metrics import mean_squared_error

print('RMSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train,y_train_pred,squared=False),
        mean_squared_error(y_test, y_test_pred,squared=False)))

RMSE train: 10.587, test: 24.795


In [15]:
from sklearn.metrics import r2_score
y_train_pred = grid.predict(x_train)
y_test_pred = grid.predict(x_test)
print('R^2 train: %.3f, test: %.3f' %
       (r2_score(y_train,y_train_pred),
        r2_score(y_test, y_test_pred)))

R^2 train: 0.982, test: 0.928
